In [1]:
import numpy as np
import argparse
import os
import random
import matplotlib.pyplot as plt
import matplotlib as mpl

RANDOM_SEED = 12345
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F

torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic=True

from utils import utils
from utils.readers import DecompensationReader
from utils.preprocessing import Discretizer, Normalizer
from utils import metrics
from utils import common_utils

from On_LSTM.model import RNNModel

In [2]:
data_path = './data/'
train_data_loader = common_utils.DeepSupervisionDataLoader(dataset_dir=os.path.join(
            data_path, 'train'), listfile=os.path.join(data_path, 'train_listfile_5%_no_val.csv'), small_part=0)

val_data_loader = common_utils.DeepSupervisionDataLoader(dataset_dir=os.path.join(
            data_path, 'train'), listfile=os.path.join(data_path, 'val_listfile_5%.csv'), small_part=0)

In [3]:
discretizer = Discretizer(timestep=1.0, store_masks=True,
                                impute_strategy='previous', start_time='zero')
discretizer_header = discretizer.transform(train_data_loader._data["X"][0])[1].split(',')
cont_channels = [i for (i, x) in enumerate(discretizer_header) if x.find("->") == -1]

normalizer = Normalizer(fields=cont_channels)
normalizer_state = 'decomp_normalizer'
normalizer_state = os.path.join(os.path.dirname(data_path), normalizer_state)
normalizer.load_params(normalizer_state)

train_data_gen = utils.BatchGenDeepSupervision(train_data_loader, discretizer,
                                                        normalizer, batch_size=128, shuffle=True, return_names=True)
val_data_gen = utils.BatchGenDeepSupervision(val_data_loader, discretizer,
                                                    normalizer, batch_size=128, shuffle=False, return_names=True)

In [4]:
'''Model structure'''
print('Constructing model ... ')
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
print("available device: {}".format(device))

model = RNNModel(rnn_type='LSTM', ntoken=76, ninp=76, nhid=384, chunk_size=128, nlayers=3,
                dropout=0.5, dropouth=0.25, dropouti=0.5, dropoute=0.1, wdrop=0, tie_weights=False)
# model = nn.LSTM(76, 76, bias=True, batch_first=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
linear = nn.Linear(76, 1)
model

Constructing model ... 
available device: cpu


RNNModel(
  (lockdrop): LockedDropout()
  (idrop): Dropout(p=0.5, inplace=False)
  (hdrop): Dropout(p=0.25, inplace=False)
  (drop): Dropout(p=0.5, inplace=False)
  (encoder): Embedding(76, 76)
  (rnn): ONLSTMStack(
    (cells): ModuleList(
      (0): ONLSTMCell(
        (ih): Sequential(
          (0): Linear(in_features=76, out_features=1542, bias=True)
        )
        (hh): LinearDropConnect(in_features=384, out_features=1542, bias=True)
      )
      (1): ONLSTMCell(
        (ih): Sequential(
          (0): Linear(in_features=384, out_features=1542, bias=True)
        )
        (hh): LinearDropConnect(in_features=384, out_features=1542, bias=True)
      )
      (2): ONLSTMCell(
        (ih): Sequential(
          (0): Linear(in_features=384, out_features=304, bias=True)
        )
        (hh): LinearDropConnect(in_features=76, out_features=304, bias=True)
      )
    )
    (lockdrop): LockedDropout()
  )
  (decoder): Linear(in_features=76, out_features=76, bias=True)
)

In [5]:
'''Train phase'''
print('Start training ... ')

train_loss = []
val_loss = []
batch_loss = []
max_auprc = 0

Start training ... 


In [10]:
for each_chunk in range(5):
    cur_batch_loss = []
    model.train()
    for each_batch in range(train_data_gen.steps):
        batch_data = next(train_data_gen)
        batch_name = batch_data['names']
        batch_data = batch_data['data']

        batch_x = torch.tensor(batch_data[0][0], dtype=torch.float32).to(device)
        batch_y = torch.tensor(batch_data[1], dtype=torch.float32).to(device)

        optimizer.zero_grad()
        loss = nn.BCELoss()

        hidden = model.init_hidden(128)
        cur_output, _ = model(batch_x, hidden)
        cur_output = torch.sigmoid(linear(cur_output))
        loss = loss(cur_output, batch_y)

        cur_batch_loss.append(loss.cpu().detach().numpy())

        loss.backward()
        optimizer.step()
        
        if each_batch % 10 == 0:
            print('Chunk %d, Batch %d: Loss = %.4f'%(each_chunk, each_batch, cur_batch_loss[-1]))

    batch_loss.append(cur_batch_loss)
    train_loss.append(np.mean(np.array(cur_batch_loss)))
    
    print("\n==>Predicting on validation")
    with torch.no_grad():
        model.eval()
        cur_val_loss = []
        valid_true = []
        valid_pred = []
        for each_batch in range(val_data_gen.steps):
            valid_data = next(val_data_gen)
            valid_name = valid_data['names']
            valid_data = valid_data['data']
            
            valid_x = torch.tensor(valid_data[0][0], dtype=torch.float32).to(device)
            valid_y = torch.tensor(valid_data[1], dtype=torch.float32).to(device)

            hidden = model.init_hidden(128)
            valid_output, _ = model(valid_x, hidden)
            valid_output = torch.sigmoid(linear(valid_output))
            valid_loss = nn.BCELoss()
            valid_loss = valid_loss(valid_output, valid_y)

            cur_val_loss.append(valid_loss.cpu().detach().numpy())

            for t, p in zip(valid_y.cpu().numpy().flatten(), valid_output.cpu().detach().numpy().flatten()):
                valid_true.append(t)
                valid_pred.append(p)

        val_loss.append(np.mean(np.array(cur_val_loss)))
        print('Valid loss = %.4f'%(val_loss[-1]))
        print('\n')
        valid_pred = np.array(valid_pred)
        valid_pred = np.stack([1 - valid_pred, valid_pred], axis=1)
        ret = metrics.print_metrics_binary(valid_true, valid_pred)
        print()

        cur_auprc = ret['auprc']
        
        if cur_auprc > max_auprc:
            max_auprc = cur_auprc
            state = {
                'net': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'chunk': each_chunk
            }
            torch.save(state, file_name)
            print('\n------------ Save best model ------------\n')

RuntimeError: The size of tensor a (1234) must match the size of tensor b (128) at non-singleton dimension 0

In [6]:
batch_data = next(train_data_gen)
batch_data = batch_data['data']



In [7]:
batch_x = torch.tensor(batch_data[0][0], dtype=torch.float32)
batch_y = torch.tensor(batch_data[1], dtype=torch.float32)
print(batch_x.shape)
print(batch_y.shape)
print(batch_x.dtype)
print(batch_y.dtype)

torch.Size([128, 1221, 76])
torch.Size([128, 1221, 1])
torch.float32
torch.float32


In [8]:
hidden = model.init_hidden(128)
print(hidden)



[(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [9]:
batch_x = batch_x.type(torch.LongTensor)
# batch_y = batch_y.type(torch.LongTensor)
cur_output, _ = model(batch_x, hidden)
cur_output.shape

RuntimeError: expected scalar type Long but found Float

In [17]:
batch_x.dtype

torch.float32

In [ ]:
linear = nn.Linear(76, 1)
cur_output = torch.sigmoid(linear(cur_output))


In [ ]:
cur_output.shape

In [ ]:
cur_output

In [ ]:
loss = nn.BCELoss()
loss = loss(cur_output, batch_y)